In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import pandas as pd
import torch
from torch.nn import CrossEntropyLoss
#from transformers import *
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizerFast, BertModel
from transformers import AlbertModel
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import ElectraModel, ElectraForMaskedLM, ElectraTokenizerFast, ElectraForPreTraining, ElectraTokenizer
from transformers import BigBirdTokenizer, BigBirdTokenizerFast, BigBirdForMaskedLM, BigBirdForCausalLM
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel, GPT2Tokenizer
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, AutoModelForMaskedLM
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import math
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
import os
import sys
logging.basicConfig(level=logging.INFO)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
custom_tokens = ['가렴주구', '각골난망', '각주구검', '간담상조', '감언이설', '개과천선', '건곤일척', '견강부회',
       '견물생심', '견원지간', '결자해지', '결초보은', '경거망동', '경국지색', '고군분투', '고육지책',
       '고장난명', '고진감래', '과유불급', '괄목상대', '구밀복검', '군계일학', '권토중래', '근묵자흑',
       '금상첨화', '금의환향', '난공불락', '난형난제', '낭중지추', '노심초사', '다다익선', '당랑거철',
       '대기만성', '독야청청', '동고동락', '동문서답', '동분서주', '동상이몽', '마부작침', '마이동풍',
       '막상막하', '망양보뢰', '맹모삼천', '명불허전', '명약관화', '문전성시', '백면서생', '복지부동',
       '부창부수', '부화뇌동', '분골쇄신', '불치하문', '비일비재', '사면초가', '사상누각', '사필귀정',
       '살신성인', '삼고초려', '상전벽해', '새옹지마', '선공후사', '설상가상', '설왕설래', '소탐대실',
       '수불석권', '수수방관', '순망치한', '아비규환', '아전인수', '어부지리', '어불성설', '언감생심',
       '여리박빙', '역지사지', '연목구어', '오리무중', '오매불망', '오비이락', '오월동주', '오합지졸',
       '온고지신', '와신상담', '외유내강', '용두사미', '용호상박', '우공이산', '유구무언', '유비무환',
       '읍참마속', '이심전심', '인과응보', '일거양득', '일구이언', '일벌백계', '일석이조', '일장춘몽',
       '일취월장', '일확천금', '자가당착', '자수성가', '자승자박', '자업자득', '전전긍긍', '전전반측',
       '전화위복', '절차탁마', '절치부심', '점입가경', '조강지처', '조변석개', '조삼모사', '주경야독',
       '주마가편', '주마간산', '죽마고우', '중구난방', '지록위마', '지리멸렬', '진퇴양난', '창해일속',
       '천의무봉', '천정부지', '천편일률', '청출어람', '초록동색', '초지일관', '촌철살인', '측은지심',
       '칠전팔기', '침소봉대', '타산지석', '토사구팽', '파죽지세', '표리부동', '풍수지탄', '풍전등화',
       '학수고대', '함흥차사', '허장성세', '형설지공', '호가호위', '혹세무민', '화룡점정', '환골탈태',
       '후생가외', '후안무치']

In [5]:
def uni_predict(text, model, tokenizer):
    # Tokenized input
    # text = "[CLS] I got restricted because Tom reported my reply [SEP]"
    text = text
    tokenized_text = tokenizer.tokenize(text)
    sentence_score = 0
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    length = len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')
    #masked_tensor = torch.tensor([masked_index])
    with torch.no_grad():
        outputs = model(tokens_tensor, labels= tokens_tensor)
    loss = outputs[0]
    sentence_score = -loss
    return sentence_score

def bert_predict(text, model, tokenizer):
    # Tokenized input
    # text = "[CLS] I got restricted because Tom reported my reply [SEP]"
    text = "[CLS] " + text + " [SEP]" #special token for BERT, RoBERTa
    tokenized_text = tokenizer.tokenize(text)
    sentence_score = 0
    length = len(tokenized_text)-2
    for masked_index in range(1,len(tokenized_text)-1):
        # Mask a token that we will try to predict back with `BertForMaskedLM`
        masked_word = tokenized_text[masked_index]
        #tokenized_text[masked_index] = '<mask>' #special token for XLNet
        tokenized_text[masked_index] = '[MASK]' #special token for BERT, RoBerta
        # Convert token to vocabulary indices
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        index = torch.tensor(tokenizer.convert_tokens_to_ids(masked_word))
        tokens_tensor = torch.tensor([indexed_tokens])
        tokens_tensor = tokens_tensor.to('cuda')
        index = index.to('cuda')
        #masked_tensor = torch.tensor([masked_index])
        with torch.no_grad():
            outputs = model(tokens_tensor)
        prediction_scores = outputs[0]
        prediction_scores = prediction_scores.view(-1, model.config.vocab_size)
        prediction_scores = prediction_scores[masked_index].unsqueeze(0)
        loss_fct = CrossEntropyLoss(ignore_index=-1)  # -1 index = padding token
        masked_lm_loss = loss_fct(prediction_scores, index.view(-1))
        tokenized_text[masked_index] = masked_word
        sentence_score -= masked_lm_loss.item()
        tokenized_text[masked_index] = masked_word
    sentence_score = sentence_score/length
    return sentence_score

def ro_predict(text, model, tokenizer):
    # Tokenized input
    # text = "[CLS] I got restricted because Tom reported my reply [SEP]"
    text = '<s> '+text+ ' </s>' #special token for RoBERTa
    tokenized_text = tokenizer.tokenize(text)
    sentence_score = 0
    length = len(tokenized_text)-2
    for masked_index in range(1,len(tokenized_text)-1):
        # Mask a token that we will try to predict back with `BertForMaskedLM`
        masked_word = tokenized_text[masked_index]
        #tokenized_text[masked_index] = '<mask>' #special token for XLNet
        tokenized_text[masked_index] = '<mask>' #special token for BERT, RoBerta
        # Convert token to vocabulary indices
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        index = torch.tensor(tokenizer.convert_tokens_to_ids(masked_word))
        tokens_tensor = torch.tensor([indexed_tokens])
        tokens_tensor = tokens_tensor.to('cuda')
        index = index.to('cuda')
        #masked_tensor = torch.tensor([masked_index])
        with torch.no_grad():
            outputs = model(tokens_tensor)
        prediction_scores = outputs[0]
        prediction_scores = prediction_scores.view(-1, model.config.vocab_size)
        prediction_scores = prediction_scores[masked_index].unsqueeze(0)
        loss_fct = CrossEntropyLoss(ignore_index=-1)  # -1 index = padding token
        masked_lm_loss = loss_fct(prediction_scores, index.view(-1))
        tokenized_text[masked_index] = masked_word
        sentence_score -= masked_lm_loss.item()
        tokenized_text[masked_index] = masked_word
    sentence_score = sentence_score/length
    return sentence_score

# BERT VERSION

In [6]:
test = sys.argv[1]
model_type = 'bert'

# 모델 선정
tokenizer = AutoTokenizer.from_pretrained('monologg/kobigbird-bert-base')
model = AutoModelForMaskedLM.from_pretrained('monologg/kobigbird-bert-base')
tokenizer.add_tokens(custom_tokens, special_tokens=True)
model.resize_token_embeddings(len(tokenizer))

model.to('cuda')
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BigBirdForMaskedLM(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32642, 768)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0-11): 12 x BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): D

In [7]:
print(tokenizer.tokenize('여기에 담백하고 부드러운 완두콩 소스까지 곁들이면 금상첨화다.'))
print(tokenizer.tokenize('부디 대기만성 게임으로 거듭나 새로운 게임 한류를 이끄는 선봉장이 됐으면 한다.'))

['여기', '##에', '담백', '##하고', '부드러운', '완', '##두', '##콩', '소스', '##까', '##지', '곁', '##들이', '##면', '금상첨화', '다', '.']
['부디', '대기만성', '게임', '##으로', '거듭나', '새로운', '게임', '한류', '##를', '이끄', '##는', '선봉', '##장이', '됐', '##으면', '한다', '.']


In [8]:
with open("/content/drive/MyDrive/기말과제_고사성어/sense_making_final.txt".format(test), "r") as f:
    file = f.readlines()
num = len(file)
count = 0
curr = 0
for line in file:
    line = line.strip().split("\001") #split("\t")
    label = int(line[0])
    score_list = []
    for sentence in line[1:]:
        if model_type=='bert':
            score = bert_predict(sentence, model=model, tokenizer=tokenizer)
        elif model_type=='roberta':
            score = ro_predict(sentence, model=model, tokenizer=tokenizer)
        else:
            score = uni_predict(sentence, model=model, tokenizer=tokenizer)
        score_list.append(score)
    #print(score_list)
    predict_label = score_list.index(max(score_list))
    #print(predict_label, label)
    if predict_label==label:
        count += 1
    curr += 1
    #print (count, curr, count/curr)
print(test+' '+model_type+':-------------------')
print (count/num)

Attention type 'block_sparse' is not possible if sequence_length: 34 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


-f bert:-------------------
0.48


In [9]:
with open("/content/drive/MyDrive/기말과제_고사성어/conjunction_acceptability_final.txt".format(test), "r") as f:
    file = f.readlines()
num = len(file)
count = 0
curr = 0
for line in file:
    line = line.strip().split("\001") #split("\t")
    label = int(line[0])
    score_list = []
    for sentence in line[1:]:
        if model_type=='bert':
            score = bert_predict(sentence, model=model, tokenizer=tokenizer)
        elif model_type=='roberta':
            score = ro_predict(sentence, model=model, tokenizer=tokenizer)
        else:
            score = uni_predict(sentence, model=model, tokenizer=tokenizer)
        score_list.append(score)
    #print(score_list)
    predict_label = score_list.index(max(score_list))
    #print(predict_label, label)
    if predict_label==label:
        count += 1
    curr += 1
    #print (count, curr, count/curr)
print(test+' '+model_type+':-------------------')
print (count/num)

-f bert:-------------------
0.87


In [10]:
predicted_list = []
label_list = []
with open("/content/drive/MyDrive/기말과제_고사성어/SWAG_final.txt", "r") as f:
    file = f.readlines()
for i in file:

        label, sentence_1, sentence_2, sentence_3 = i.split("\001")
        prob_1 = bert_predict(sentence_1, model=model, tokenizer=tokenizer)
        prob_2 = bert_predict(sentence_2, model=model, tokenizer=tokenizer)
        prob_3 = bert_predict(sentence_3, model=model, tokenizer=tokenizer)

        score_list = [prob_1, prob_2, prob_3]
        predict_label = score_list.index(max(score_list))
        predicted_list.append(predict_label)
        label_list.append(int(label[0]))
print(predicted_list)
print(label_list)

[0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 0, 2, 2, 1, 2, 1, 1, 0, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 1, 0, 2, 1, 1, 2, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 0, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [11]:
RESULT = [i for i, j in zip(label_list, predicted_list) if i == j]
len(RESULT)/100

0.64

# RoBERTa VERSION

In [13]:
test = sys.argv[1]
model_type = 'roberta'

# 모델 선정
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
model = AutoModelForMaskedLM.from_pretrained("klue/roberta-large")
tokenizer.add_tokens(custom_tokens, special_tokens=True)
model.resize_token_embeddings(len(tokenizer))

model.to('cuda')
model.eval()

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32141, 1024)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): Lay

In [14]:
print(tokenizer.tokenize('여기에 담백하고 부드러운 완두콩 소스까지 곁들이면 금상첨화다.'))
print(tokenizer.tokenize('부디 대기만성 게임으로 거듭나 새로운 게임 한류를 이끄는 선봉장이 됐으면 한다.'))

['여기', '##에', '담백', '##하고', '부드러운', '완', '##두', '##콩', '소스', '##까', '##지', '곁들이', '##면', '금상첨화', '다', '.']
['부디', '대기만성', '게임', '##으로', '거듭나', '새로운', '게임', '한류', '##를', '이끄', '##는', '선봉', '##장이', '됐', '##으면', '한다', '.']


In [15]:
with open("/content/drive/MyDrive/기말과제_고사성어/sense_making_final.txt".format(test), "r") as f:
    file = f.readlines()
num = len(file)
count = 0
curr = 0
for line in file:
    line = line.strip().split("\001")
    label = int(line[0])
    score_list = []
    for sentence in line[1:]:
        if model_type=='bert':
            score = bert_predict(sentence, model=model, tokenizer=tokenizer)
        elif model_type=='roberta':
            score = ro_predict(sentence, model=model, tokenizer=tokenizer)
        else:
            score = uni_predict(sentence, model=model, tokenizer=tokenizer)
        score_list.append(score)
    #print(score_list)
    predict_label = score_list.index(max(score_list))
    #print(predict_label, label)
    if predict_label==label:
        count += 1
    curr += 1
    #print (count, curr, count/curr)
print(test+' '+model_type+':-------------------')
print (count/num)

-f roberta:-------------------
0.61


In [16]:
with open("/content/drive/MyDrive/기말과제_고사성어/conjunction_acceptability_final.txt".format(test), "r") as f:
    file = f.readlines()
num = len(file)
count = 0
curr = 0
for line in file:
    line = line.strip().split("\001") #split("\t")
    label = int(line[0])
    score_list = []
    for sentence in line[1:]:
        if model_type=='bert':
            score = bert_predict(sentence, model=model, tokenizer=tokenizer)
        elif model_type=='roberta':
            score = ro_predict(sentence, model=model, tokenizer=tokenizer)
        else:
            score = uni_predict(sentence, model=model, tokenizer=tokenizer)
        score_list.append(score)
    #print(score_list)
    predict_label = score_list.index(max(score_list))
    #print(predict_label, label)
    if predict_label==label:
        count += 1
    curr += 1
    #print (count, curr, count/curr)
print(test+' '+model_type+':-------------------')
print (count/num)

-f roberta:-------------------
0.42


In [17]:
predicted_list = []
label_list = []
with open("/content/drive/MyDrive/기말과제_고사성어/SWAG_final.txt", "r") as f:
    file = f.readlines()
for i in file:

        label, sentence_1, sentence_2, sentence_3 = i.split("\001")
        prob_1 = ro_predict(sentence_1, model=model, tokenizer=tokenizer)
        prob_2 = ro_predict(sentence_2, model=model, tokenizer=tokenizer)
        prob_3 = ro_predict(sentence_3, model=model, tokenizer=tokenizer)

        score_list = [prob_1, prob_2, prob_3]
        predict_label = score_list.index(max(score_list))
        predicted_list.append(predict_label)
        label_list.append(int(label[0]))
print(predicted_list)
print(label_list)

[0, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 2, 2, 1, 0, 1, 1, 0, 0, 0, 0, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 1, 0, 2, 1, 2, 2, 0, 1, 0, 1, 2, 1, 1, 2, 2, 1, 1, 0, 0, 1, 1, 1, 0, 2, 2, 0, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, 2, 0, 1, 2, 2, 0, 1, 2, 1, 0, 2, 2, 1, 2, 2, 1, 2, 0, 2, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [18]:
RESULT = [i for i, j in zip(label_list, predicted_list) if i == j]
len(RESULT)/100

0.37

# GPT VERSION

In [19]:
test = sys.argv[1]
model_type = 'gpt'

# 모델 선정
tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
model = GPT2LMHeadModel.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
tokenizer.add_tokens(custom_tokens, special_tokens=True)
model.resize_token_embeddings(len(tokenizer))

model.to('cuda')
model.eval()

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51346, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1920, out_features=51346, bias=False)
)

In [20]:
with open("/content/drive/MyDrive/기말과제_고사성어/sense_making_final.txt".format(test), "r") as f:
    file = f.readlines()
num = len(file)
count = 0
curr = 0
for line in file:
    line = line.strip().split("\001")
    label = int(line[0])
    score_list = []
    for sentence in line[1:]:
        if model_type=='bert':
            score = bert_predict(sentence, model=model, tokenizer=tokenizer)
        elif model_type=='roberta':
            score = ro_predict(sentence, model=model, tokenizer=tokenizer)
        else:
            score = uni_predict(sentence, model=model, tokenizer=tokenizer)
        score_list.append(score)
    #print(score_list)
    predict_label = score_list.index(max(score_list))
    #print(predict_label, label)
    if predict_label==label:
        count += 1
    curr += 1
    #print (count, curr, count/curr)
print(test+' '+model_type+':-------------------')
print (count/num)

-f gpt:-------------------
0.47


In [21]:
with open("/content/drive/MyDrive/기말과제_고사성어/conjunction_acceptability_final.txt".format(test), "r") as f:
    file = f.readlines()
num = len(file)
count = 0
curr = 0
for line in file:
    line = line.strip().split("\001") #split("\t")
    label = int(line[0])
    score_list = []
    for sentence in line[1:]:
        if model_type=='bert':
            score = bert_predict(sentence, model=model, tokenizer=tokenizer)
        elif model_type=='roberta':
            score = ro_predict(sentence, model=model, tokenizer=tokenizer)
        else:
            score = uni_predict(sentence, model=model, tokenizer=tokenizer)
        score_list.append(score)
    #print(score_list)
    predict_label = score_list.index(max(score_list))
    #print(predict_label, label)
    if predict_label==label:
        count += 1
    curr += 1
    #print (count, curr, count/curr)
print(test+' '+model_type+':-------------------')
print (count/num)

-f gpt:-------------------
0.86


In [22]:
predicted_list = []
label_list = []
with open("/content/drive/MyDrive/기말과제_고사성어/SWAG_final.txt", "r") as f:
    file = f.readlines()
for i in file:

        label, sentence_1, sentence_2, sentence_3 = i.split("\001")
        prob_1 = uni_predict(sentence_1, model=model, tokenizer=tokenizer)
        prob_2 = uni_predict(sentence_2, model=model, tokenizer=tokenizer)
        prob_3 = uni_predict(sentence_3, model=model, tokenizer=tokenizer)

        score_list = [prob_1, prob_2, prob_3]
        predict_label = score_list.index(max(score_list))
        predicted_list.append(predict_label)
        label_list.append(int(label[0]))
print(predicted_list)
print(label_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 0, 2, 2, 1, 0, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [23]:
RESULT = [i for i, j in zip(label_list, predicted_list) if i == j]
len(RESULT)/100

0.81